## Installation des outils

In [0]:
# On installe la librairie Prophet (Facebook) et Pandas
%pip install prophet pandas

dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/50.4 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 41.0/50.4 kB 5.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 807.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/12.1 MB ? eta -:--:--
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/12.1 MB 71.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 6.6/12.1 MB 96.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 11.2/12.1 MB 113.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 12.1/12.1 MB 112.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 12.1/12.1 MB 112.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 12.1/12.1 MB 112.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.2 MB/s et

## 1. Connexion et Configuration

In [0]:
from pyspark.sql.functions import col, to_timestamp, input_file_name, regexp_extract, lit
from pyspark.sql.types import *
import pandas as pd
from prophet import Prophet
import logging

# On cache les messages techniques inutiles de Prophet
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

# --- IDENTIFIANTS ---
storage_account_name = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  
storage_account_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Connexion sécurisée
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

print(" Connexion établie.")

🔌 Connexion établie.


## 2. LECTURE DE LA TABLE SILVER (CLEAN)

In [0]:
# =============================================================================
# 1. LECTURE DE LA TABLE SILVER (CLEAN)
# =============================================================================
# On ne lit plus le JSON, on lit la table Delta propre qu'on vient de créer
path_silver = f"abfss://clean@{storage_account_name}.dfs.core.windows.net/silver_le_havre_complete"
df_silver = spark.read.format("delta").load(path_silver)
pdf = df_silver.toPandas()
# 1. On recharge la table qu'on vient de sauvegarder
df_check = spark.read.format("delta").load(path_silver)

# 2. On compte les lignes par type
print("📊 VÉRIFICATION DU CONTENU :")
print(df_check.count())

📊 VÉRIFICATION DU CONTENU :
3168


## PRÉDIRE LA POLLUTION 

In [0]:
df_prophet = pdf.rename(columns={'date_heure': 'ds', 'pm25_value': 'y'})

# Hna l'astuce:
# df_train: Fih gher l'historique (fin 3ndna valeur d pollution) -> Bach ntriniw
# df_context: Fih kolchi (historique + futur meteo) -> Bach ndiro prediction
df_train = df_prophet[df_prophet['y'].notna()].copy()

print(f"📊 Training sur {len(df_train)} lignes | Prédiction sur {len(df_prophet)} lignes")

# =============================================================================
# 2. ENTRAÎNEMENT DU MODÈLE (Prophet)
# =============================================================================
m_pol = Prophet(daily_seasonality=True)
m_pol.add_country_holidays(country_name='FR') 

# Ajout des Regresseurs (Meteo)
m_pol.add_regressor('temperature')
m_pol.add_regressor('vent_vitesse')
m_pol.add_regressor('humidite')
m_pol.add_regressor('precipitation')

# Zid direction ila kant (Muhima bzaf)
if 'vent_direction' in df_train.columns:
    m_pol.add_regressor('vent_direction')

print("🏋️ Entraînement du modèle en cours...")
m_pol.fit(df_train)

# =============================================================================
# 3. PRÉDICTION GLOBALE (SUR LA VRAIE MÉTÉO FUTURE)
# =============================================================================
forecast_pol = m_pol.predict(df_prophet)

# Fusion
results = pd.merge(forecast_pol[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], df_prophet, on='ds', how='left')

# =============================================================================
# 4. MARQUAGE (ANOMALIES & PRÉVISIONS)
# =============================================================================
def identifier_statut(row):
    if pd.isna(row['y']):
        return "PRÉVISION"
    # Ila y kber mn tawa9o3 lfa9ani -> Anomalie
    elif row['y'] > row['yhat_upper']:
        return "ANOMALIE HAUTE"
    # Ila y sghar mn tawa9o3 ta7tani -> Anomalie
    elif row['y'] < row['yhat_lower']:
        return "ANOMALIE BASSE"
    else:
        return "Normal"

results['statut'] = results.apply(identifier_statut, axis=1)

# =============================================================================
# 5. EXPORT FINAL VERS GOLD
# =============================================================================
cols_to_keep = ['ds', 'y', 'yhat', 'yhat_lower', 'yhat_upper', 'statut', 
                'temperature', 'vent_vitesse', 'humidite', 'precipitation']

# 1. Définition du Schema (Bach Spark maydoukhch m3a les Nulls)
schema_fields = [
    StructField("ds", TimestampType(), True),
    StructField("y", DoubleType(), True),
    StructField("yhat", DoubleType(), True),
    StructField("yhat_lower", DoubleType(), True),
    StructField("yhat_upper", DoubleType(), True),
    StructField("statut", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("vent_vitesse", DoubleType(), True),
    StructField("humidite", DoubleType(), True),
    StructField("precipitation", DoubleType(), True)
]

if 'vent_direction' in results.columns:
    schema_fields.append(StructField("vent_direction", DoubleType(), True))
    if 'vent_direction' not in cols_to_keep:
        cols_to_keep.append('vent_direction')

schema = StructType(schema_fields)

# 2. Création du DataFrame Spark avec Schema Explicite

df_spark_final = spark.createDataFrame(
    results[cols_to_keep],
    schema=schema
)

# 3. Sauvegarde Delta
path_gold = f"abfss://curated@{storage_account_name}.dfs.core.windows.net/gold_pollution_complete"
df_spark_final.write.format("delta").mode("overwrite").save(path_gold)

print(f"🏆 TERMINE ! Gold Layer disponible ici : {path_gold}")
display(df_spark_final.orderBy("ds", ascending=False).limit(24))

📊 Training sur 2322 lignes | Prédiction sur 3168 lignes
🏋️ Entraînement du modèle en cours...


17:13:42 - cmdstanpy - INFO - Chain [1] start processing
17:13:43 - cmdstanpy - INFO - Chain [1] done processing


🏆 TERMINE ! Gold Layer disponible ici : abfss://curated@sadatalakeair2025.dfs.core.windows.net/gold_pollution_complete


ds,y,yhat,yhat_lower,yhat_upper,statut,temperature,vent_vitesse,humidite,precipitation,vent_direction
2026-01-28T23:00:00Z,null,7.73289443700833,2.071408798015048,13.081730892191695,🔮 PRÉVISION (15j),-1.8,34.0,53.0,0.0,80.0
2026-01-28T22:00:00Z,null,7.684156147208181,1.9096993178848705,13.250376908424554,🔮 PRÉVISION (15j),-1.4,34.6,52.0,0.0,81.0
2026-01-28T21:00:00Z,null,7.54280974808431,2.1717226376526626,13.21420446962456,🔮 PRÉVISION (15j),-0.9,35.3,52.0,0.0,81.0
2026-01-28T20:00:00Z,null,7.295769249289414,1.7551473972678433,12.839896226515856,🔮 PRÉVISION (15j),-0.5,36.1,51.0,0.0,81.0
2026-01-28T19:00:00Z,null,6.958414143753799,1.2103923751414463,12.320914946683423,🔮 PRÉVISION (15j),-0.0,36.5,51.0,0.0,80.0
2026-01-28T18:00:00Z,null,6.526773321125634,0.9859622741179953,12.121941762172781,🔮 PRÉVISION (15j),0.5,36.9,51.0,0.0,80.0
2026-01-28T17:00:00Z,null,6.115739467429842,0.5458627437375064,11.322984178268726,🔮 PRÉVISION (15j),1.0,37.0,50.0,0.0,79.0
2026-01-28T16:00:00Z,null,5.824671809100724,0.18436170549226694,11.199308212238101,🔮 PRÉVISION (15j),1.4,36.9,50.0,0.0,80.0
2026-01-28T15:00:00Z,null,5.7807264067325,0.15100959796281527,11.206549476476775,🔮 PRÉVISION (15j),1.6,36.4,49.0,0.0,81.0
2026-01-28T14:00:00Z,null,5.957462603353122,0.49512656371874764,11.53992699835455,🔮 PRÉVISION (15j),1.6,35.9,48.0,0.0,84.0


In [1]:
display(df_spark_final)

NameError: name 'df_spark_final' is not defined